# How to break into the field
In this notebook I will follow along the nb from Udacity course and practice some basic data wrangling. <br>
**Question: What do the survey takers advise to break into the field of software development?**

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('survey-results-public.csv')
df.head(2)

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0


To answer the question, we need to check out the CousinEducation column in the schema.csv file.

In [4]:
dfs = pd.read_csv('survey-results-schema.csv')
list(dfs[dfs.Column == 'CousinEducation']['Question'])

["Let's pretend you have a distant cousin. They are 24 years old, have a college degree in a field not related to computer programming, and have been working a non-coding job for the last two years. They want your advice on how to switch to a career as a software developer. Which of the following options would you most strongly recommend to your cousin?\nLet's pretend you have a distant cousin named Robert. He is 24 years old, has a college degree in a field not related to computer programming, and has been working a non-coding job for the last two years. He wants your advice on how to switch to a career as a software developer. Which of the following options would you most strongly recommend to Robert?\nLet's pretend you have a distant cousin named Alice. She is 24 years old, has a college degree in a field not related to computer programming, and has been working a non-coding job for the last two years. She wants your advice on how to switch to a career as a software developer. Which

In [5]:
# Just checking what portion of this column is NaN.
df.CousinEducation.isnull().mean()

0.5414072229140723

In [6]:
# Now let's check the distribution of different answers.
# Note that reset_index() converts pd series to dataframe
study = df['CousinEducation'].value_counts().reset_index()
study.head()

,index,CousinEducation
0,Take online courses; Buy books and work throug...,711
1,Take online courses,551
2,None of these,523
3,Take online courses; Part-time/evening courses...,479
4,Take online courses; Bootcamp; Part-time/eveni...,465


Looks like this is a multiple choice question with many possible combinations, separated by ; <br>
Let's try to clean this up.


In [7]:
# First let's change the column names
#study.rename(columns = {'index': 'method', 'CousinEducation': 'count'}, inplace = True)

# in this case since we're renaming all the columns so it's more convenient this way:
study.columns = ['method', 'count']
study.head(2)

,method,count
0,Take online courses; Buy books and work throug...,711
1,Take online courses,551


### Write a function to clean up the data in the df study
Here I will try to write a function which is **slightly more efficient than the example provided in the course material**. Instead of searching for all the possible methods in each row of the df, I will loop through all the rows in the df, at each row convert the 'Method' str into a list of different methods. Then go through that list, and += the count to the corresponding method in the dict.

In [27]:
def get_count(df, cola, colb):
    """
    INPUT: 
    df - The dataframe you want to search counts from
    cola - (str) The name of the col where the keys (categories) are
    colb - (str) The nmae of the col where the count numbers are
    
    OUTPUT:
    count - A dataframe showing the count number of each key (category)
    
    """
    count = defaultdict(int)
    for i in range(df.shape[0]):
        s = df[cola][i]
        keys = s.split('; ')
        for key in keys:
            key = key.strip()
            count[key] += df[colb][i]
    count = pd.DataFrame(pd.Series(count)).reset_index()
    count.columns = [cola, colb]
    count.sort_values(colb, ascending = False, inplace = True)
    count['perc'] = count[colb] / np.sum(count[colb])
    
    return count 

In [28]:
study = get_count(study, 'method', 'count')

study.head()

,method,count,perc
0,Take online courses,15246,0.209432
1,Buy books and work through the exercises,11750,0.161408
3,Part-time/evening courses,7517,0.103260
7,Contribute to open source,7423,0.101968
4,Bootcamp,5276,0.072476


### Calculate the average salary of people who suggested each Break-in-the-field method
We can modify the function written above, to add two more dataframes and merge all three as return. The addional dataframes will contain information needed to calculate the average salary and standard deviation of each method group.

In [13]:
def get_count_all(df, cat_col, avg_col):
    """
    INPUT: 
    df - The dataframe you want to search counts from
    cat_col - (str) The name of the col where the keys (categories) are
    avg_col - (str) The nmae of the col where the average is to be calculated (e.g., salary or job satisfaction)
    
    OUTPUT:
    count - A dataframe showing the count number of each key (category)
    
    """
    count = defaultdict(int)
    total = defaultdict(int)
    sq_total = defaultdict(int)
    
    for i in range(df.shape[0]):
        # we only consider those with data in avg_col:
        if df[cat_col].isnull()[i] or df[avg_col].isnull()[i]:
            continue
        
        # go through each eligible row and collect data in dicts:
        s = df[cat_col][i]
        keys = s.split('; ')
        for key in keys:
            key = key.strip()
            count[key] += 1
            money = df[avg_col][i]
            total[key] += money
            sq_total[key] += (money ** 2)
    
    # convert the 3 dicts into dataframes with category as index
    # resetting index can make sure the indices are sorted to same order in 3 df's?
    count = pd.DataFrame(pd.Series(count)).reset_index()
    total = pd.DataFrame(pd.Series(total)).reset_index()
    sq_total = pd.DataFrame(pd.Series(sq_total)).reset_index()
    
    # change the column names of each df
    count.columns = [cat_col, 'ppl_count']
    total.columns = [cat_col, 'sum']
    sq_total.columns = [cat_col, 'sq_sum']
    
    # merge dataframes
    df_means = pd.merge(total, count)
    df_all = pd.merge(df_means, sq_total)
            
    # additional cols for statistic analyses
    df_all['mean_col'] = df_means['sum'] / df_means['ppl_count']
    df_all['variance'] = df_all['sq_sum'] / df_all['ppl_count'] - (df_all['mean_col'] ** 2)
    df_all['std'] = np.sqrt(df_all['variance'])
    df_all['lower_95'] = df_all['mean_col'] - 1.96 * df_all['std'] / np.sqrt(df_all['ppl_count'])
    df_all['higher_95'] = df_all['mean_col'] + 1.96 * df_all['std'] / np.sqrt(df_all['ppl_count'])
    
    return df_all

In [15]:
df_all = get_count_all(df, 'CousinEducation', 'Salary')
df_all.sort_values('mean_col', ascending=False)

,CousinEducation,sum,ppl_count,sq_sum,mean_col,variance,std,lower_95,higher_95
8,Contribute to open source,1.392267e+08,2255,1.239062e+13,61741.337383,1.682738e+09,41021.185497,60048.204652,63434.470114
10,Other,4.491415e+07,738,3.851360e+12,60859.281694,1.514792e+09,38920.331073,58051.234411,63667.328977
12,Master's degree,4.284612e+07,721,3.771773e+12,59425.969277,1.699862e+09,41229.387609,56416.462517,62435.476037
11,Return to college,8.733691e+07,1474,7.243713e+12,59251.636145,1.403567e+09,37464.208781,57339.037276,61164.235014
6,Bootcamp,9.583229e+07,1623,8.502989e+12,59046.391551,1.752580e+09,41863.829121,57009.650764,61083.132338
9,Participate in hackathons,4.641498e+07,797,4.044473e+12,58237.114855,1.683060e+09,41025.111976,55388.874170,61085.355540
1,Conferences/meet-ups,9.699603e+07,1679,8.366275e+12,57770.118326,1.645505e+09,40564.823281,55829.767269,59710.469383
0,Get a job as a QA tester,5.852363e+07,1033,5.017250e+12,56654.043442,1.647290e+09,40586.814900,54178.954092,59129.132792
4,Buy books and work through the exercises,1.909928e+08,3395,1.624985e+13,56257.071544,1.621550e+09,40268.471257,54902.501945,57611.641144
2,Take online courses,2.415638e+08,4495,2.011544e+13,53740.566104,1.587022e+09,39837.444455,52575.950260,54905.181948


It seems like "Contribute to open source" is the category with the highest average salary. However, the 95% confidence interval of adjacent rows are mostly overlapped. So it's hard to say that, for example, contributing to open source is much more effective to get a high salary compared to getting a Master's degree.<br>
Let's do the same analysis with the column "JobSatisfaction":

In [16]:
df_job_satis = get_count_all(df, 'CousinEducation', 'JobSatisfaction')
df_job_satis.sort_values('mean_col', ascending=False)

,CousinEducation,sum,ppl_count,sq_sum,mean_col,variance,std,lower_95,higher_95
12,Participate in hackathons,14884.0,2107,115166.0,7.064072,4.757641,2.181202,6.970936,7.157209
11,Return to college,27767.0,3943,212971.0,7.042100,4.421256,2.102678,6.976468,7.107732
0,Get a job as a QA tester,21294.0,3025,162716.0,7.039339,4.238122,2.058670,6.965975,7.112702
7,Other,13797.0,1961,106521.0,7.035696,4.818716,2.195157,6.938537,7.132855
3,Contribute to open source,42374.0,6060,324340.0,6.992409,4.627665,2.151201,6.938246,7.046572
9,Bootcamp,30404.0,4351,231670.0,6.987819,4.415618,2.101337,6.925380,7.050258
1,Conferences/meet-ups,30868.0,4419,236106.0,6.985291,4.635448,2.153009,6.921810,7.048771
4,Buy books and work through the exercises,66788.0,9580,508944.0,6.971608,4.522367,2.126586,6.929023,7.014192
2,Take online courses,85746.0,12337,651056.0,6.950312,4.465797,2.113243,6.913021,6.987603
5,Part-time/evening courses,42797.0,6164,324829.0,6.943056,4.491728,2.119370,6.890147,6.995966


The above results look even more ambiguous. It's really hard to tell if there's any big difference in terms of Job Satisfaction.

### Check if there's any bias towards their own degree 
We will write a function to determine if a respondent has a higher degree. But first let's check what degree categories are included in this survey.

In [23]:
degree_count = df.FormalEducation.value_counts().reset_index()
degree_count['precentage'] = degree_count['FormalEducation'] / np.sum(degree_count['FormalEducation'])
degree_count

,index,FormalEducation,precentage
0,Bachelor's degree,21609,0.420474
1,Master's degree,11141,0.216785
2,Some college/university study without earning ...,8129,0.158176
3,Secondary school,5908,0.114960
4,Doctoral degree,1308,0.025451
5,I prefer not to answer,1109,0.021579
6,Primary/elementary school,1047,0.020373
7,Professional degree,715,0.013913
8,I never completed any formal education,426,0.008289


In [20]:
def higher_ed(formal_ed_str):
    return 1 if formal_ed_str in {'Master\'s degree', 'Professional degree', 'Doctoral degree'} else 0

# check if this function works
df.FormalEducation.apply(higher_ed)[:5]

0    0
1    0
2    0
3    1
4    1
Name: FormalEducation, dtype: int64

In [22]:
# Now we can create a new column in the df to indicate if this respondent has higher degree.
df['HigherEd'] = df.FormalEducation.apply(higher_ed)
higher_ed_perc = df.HigherEd.mean()
higher_ed_perc

0.25614881693648817

In [26]:
# Extract the rows in the df where ‘HigherEd’ is 1 or 0:
ed1 = df[df['HigherEd'] == 1]
ed0 = df[df['HigherEd'] == 0]

# Check if the new df's look right:
print(ed1['HigherEd'][:5])
ed0['HigherEd'][:5]

3    1
4    1
6    1
7    1
9    1
Name: HigherEd, dtype: int64


0    0
1    0
2    0
5    0
8    0
Name: HigherEd, dtype: int64

In [37]:
# Use the two new dfs to calculate the percentage of each CousinEducation category, and see if there's any bias
ed1_count = ed1.CousinEducation.value_counts().reset_index()
ed0_count = ed0.CousinEducation.value_counts().reset_index()
ed1_count.columns = ['method', 'count']
ed0_count.columns = ['method', 'count']

In [53]:
ed1_perc = get_count(ed1_count, 'method', 'count').drop('count', axis = 1)
ed1_perc.rename(columns = {'perc': 'ed_1_perc'}, inplace = True)
ed0_perc = get_count(ed0_count, 'method', 'count').drop('count', axis = 1)
ed0_perc.rename(columns = {'perc': 'ed_0_perc'}, inplace = True)

# Merge to make a new df and compare the difference, plot the difference
comp_df = pd.merge(ed1_perc, ed0_perc)
comp_df['Diff_HigherEd_Vals'] = comp_df['ed_1_perc'] - comp_df['ed_0_perc']
comp_df.style.bar(subset = ['Diff_HigherEd_Vals'], align = 'mid', color = ['#d65f5f', '#5fba7d'])

,method,ed_1_perc,ed_0_perc,Diff_HigherEd_Vals
0,Take online courses,0.206234,0.210548,-0.004314
1,Buy books and work through the exercises,0.162959,0.160867,0.002092
2,Part-time/evening courses,0.101736,0.103791,-0.002055
3,Contribute to open source,0.100409,0.102513,-0.002104
4,Bootcamp,0.074762,0.071677,0.003085
5,Return to college,0.068975,0.068898,0.000077
6,Conferences/meet-ups,0.064461,0.074679,-0.010218
7,Master's degree,0.060532,0.027778,0.032754
8,Get a job as a QA tester,0.047257,0.046068,0.001190
9,Participate in online coding competitions,0.045027,0.051182,-0.006155
